In [1]:
import tensorflow as tf
import cv2, numpy as np, os
import pandas as pd, time
import tensorflow.contrib as tf_contrib

DECAY_BN = 0.0001
EPSILON_BN = 1e-05

In [2]:
def read_images(path, ran, phase):
    save_path = phase+"_images.npy"
    if(os.path.exists(save_path)):
        print("npy file available...")
        return np.load(save_path)
    images = []
    for r, d, f in os.walk(path):
        for i in range(1, ran+1):
            img_name = os.path.join(r, str(i)+".jpg")
            images.append(cv2.imread(img_name)/255.)
    print("done loading from " + path)
    images = np.array(images)
    np.save(save_path, images)
    return images

def read_labels(csv_file, phase, nb_classes = 8):
    
    labels = np.array(pd.read_csv(csv_file, header = None).values)
    labels = np.reshape(labels, (labels.shape[1], labels.shape[0]))
    one_hot_targets = np.eye(nb_classes)[[i-1 for i in labels]]
    one_hot_targets = np.reshape(one_hot_targets, (one_hot_targets.shape[0], 8))

    return one_hot_targets
    

def get_num_trainable_params():
    total_parameters = 0
    for variable in tf.trainable_variables():
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        total_parameters += variable_parameters
    print(total_parameters)

In [3]:
train_images = read_images("../train/", 1888, "train")
test_images = read_images("../test/", 800, "test")

train_labels = read_labels("../train_labels.csv", "train")
test_labels = read_labels("../test_labels.csv", "test")

print("Train images: " + str(train_images.shape))
print("Test images:  " + str(test_images.shape))

print("Train labels: " + str(train_labels.shape))
print("Test labels:  " + str(test_labels.shape))

npy file available...
npy file available...
Train images: (1888, 256, 256, 3)
Test images:  (800, 256, 256, 3)
Train labels: (1888, 8)
Test labels:  (800, 8)


In [4]:
def residual_block(x, filter_size, wt_init, weight_regularizer, conv_number, instance, is_train, stride = 1, reuse = False):
    
    prev = x
    x = tf.layers.conv2d(x, filters=filter_size, kernel_size=3, kernel_initializer=wt_init, strides=stride,
                                kernel_regularizer=weight_regularizer, padding="SAME", trainable=is_train, name="shortcut_conv"+conv_number+"_"+instance, reuse=reuse)
    x = tf_contrib.layers.batch_norm(x, decay=DECAY_BN, epsilon=EPSILON_BN, center=True, scale=True, trainable=is_train, reuse=reuse, scope="short_batch_norm"+conv_number+"_"+instance)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, filters=filter_size, kernel_size=3, kernel_initializer=wt_init, trainable=is_train,
                         kernel_regularizer=weight_regularizer, name="conv"+conv_number+"_"+instance, padding="SAME", reuse=reuse)
    
    x = tf_contrib.layers.batch_norm(x, decay=DECAY_BN, epsilon=EPSILON_BN, center=True, scale=True, trainable=is_train, reuse=reuse, scope="batch_norm"+conv_number+"_"+instance)
    
    if(stride == 2):
        #x = prev + x
        prev = tf.layers.conv2d(prev, filters=filter_size, kernel_size=1, kernel_initializer=wt_init, strides=stride,
                                kernel_regularizer=weight_regularizer, padding="SAME", trainable=is_train, name="1x1_"+conv_number+"_"+instance, reuse=reuse)
    
    x = prev + x
    x = tf.nn.relu(x)
    
    return x

def build_model(train_input, is_train = False, reuse = False):
    wt_init = tf_contrib.layers.variance_scaling_initializer()
    weight_regularizer = tf_contrib.layers.l2_regularizer(0.0001)
    
    x = tf.layers.conv2d(train_input, filters=64, kernel_size=7, kernel_initializer=wt_init, strides=2, 
                         kernel_regularizer=weight_regularizer, trainable=is_train, name="conv1", padding='SAME', reuse=reuse)
    x = tf_contrib.layers.batch_norm(x, decay=DECAY_BN, epsilon=EPSILON_BN, center=True, scale=True, trainable=is_train, reuse=reuse, scope="batch_norm1")
    x = tf.nn.relu(x)
    
    x = tf.layers.max_pooling2d(x, pool_size = 2, strides = 2, name="maxpool") # 56x56x64
    
    x = residual_block(x, 64, wt_init, weight_regularizer, "2", "1", is_train, reuse=reuse)
    x = residual_block(x, 64, wt_init, weight_regularizer, "2", "2", is_train, reuse=reuse)
    
    x = residual_block(x, 128, wt_init, weight_regularizer, "3", "1", is_train, stride=2, reuse=reuse)
    x = residual_block(x, 128, wt_init, weight_regularizer, "3", "2", is_train, reuse=reuse)
    
    x = residual_block(x, 256, wt_init, weight_regularizer, "4", "1", is_train, stride=2, reuse=reuse)
    x = residual_block(x, 256, wt_init, weight_regularizer, "4", "2", is_train, reuse=reuse)
    
    x = residual_block(x, 512, wt_init, weight_regularizer, "5", "1", is_train, stride=2, reuse=reuse)
    x = residual_block(x, 512, wt_init, weight_regularizer, "5", "2", is_train, reuse=reuse)
    
    x = tf.layers.average_pooling2d(x, pool_size=7, strides=2, name="avgpool")
    x = tf.contrib.layers.flatten(x)
    
    #x = tf.layers.dense(x, 1000, activation=tf.nn.relu, kernel_initializer=wt_init, trainable=is_train, name="fc1")
    x = tf.layers.dense(x, 8, kernel_initializer=wt_init, trainable=is_train, name="output", reuse=reuse)
    #x = tf.nn.softmax(x, name="output_logits")
    
    return x

def get_losses(output, label):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=label))
    prediction = tf.equal(tf.argmax(output, -1), tf.argmax(label, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    
    return loss, accuracy

In [5]:
tf.reset_default_graph()
train_input = tf.placeholder(tf.float32, (None, 224, 224, 3), name = "train_input")
train_label = tf.placeholder(tf.int32, (None, 8), name = "train_label")
test_input = tf.placeholder(tf.float32, (None, 224, 224, 3), name = "test_input")
test_label = tf.placeholder(tf.int32, (None, 8), name = "test_label")

batch = 32

lr = tf.placeholder(tf.float32, name='learning_rate')
train_output = build_model(train_input, is_train = True)
test_output = build_model(test_input, reuse = True)

train_loss, train_acc = get_losses(train_output, train_label)
test_loss, test_acc = get_losses(test_output, test_label)

optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9).minimize(train_loss)

#### Summary #####
summary_train_loss = tf.summary.scalar("train_loss", train_loss)
summary_test_loss = tf.summary.scalar("train_loss", test_loss)

summary_train_accuracy = tf.summary.scalar("train_accuracy", train_acc)
summary_test_accuracy = tf.summary.scalar("train_accuracy", test_acc)

train_summary = tf.summary.merge([summary_train_loss, summary_train_accuracy])
test_summary = tf.summary.merge([summary_test_loss, summary_test_accuracy])

#get_num_trainable_params()

In [6]:
def get_mini_batches(X, y, crop = True, batch_size = 32):
    
    n_minibatches = X.shape[0] // batch_size
    data = list(zip(X, y))
    np.random.shuffle(data)
    X, y = zip(*data)
    X = np.array(X)
    y = np.array(y)
    mini_batches = []
    i = 0
    for i in range(n_minibatches):
        X_mini = X[i * batch_size:(i + 1)*batch_size]
        if(crop):
            X_mini = data_augmentation(X_mini, 224)
        y_mini = y[i * batch_size:(i + 1)*batch_size]
        mini_batches.append((X_mini, y_mini))
    if(X.shape[0] % batch_size != 0):
        X_mini = X[i * batch_size:X.shape[0]]
        if(crop):
            X_mini = data_augmentation(X_mini, 224)
        y_mini = y[i * batch_size:y.shape[0]]
        mini_batches.append((X_mini, y_mini))
        n_minibatches += 1
    return mini_batches, n_minibatches

def _random_crop(batch, crop_shape, padding=None):
    
    oshape = np.shape(batch[0])
    if padding:
        oshape = (oshape[0] + 2 * padding, oshape[1] + 2 * padding)
    new_batch = []
    npad = ((padding, padding), (padding, padding), (0, 0))
    for i in range(len(batch)):
        new_batch.append(batch[i])
        if padding:
            new_batch[i] = np.lib.pad(batch[i], pad_width=npad, mode='constant', constant_values=0)
        nh = np.random.randint(0, oshape[0] - crop_shape[0])
        nw = np.random.randint(0, oshape[1] - crop_shape[1])
        new_batch[i] = new_batch[i][nh:nh + crop_shape[0], nw:nw + crop_shape[1]]
    return new_batch
        
    
def data_augmentation(batch, img_size):
    return _random_crop(batch, [img_size, img_size, 3])


aug_test_images = data_augmentation(test_images, 224)

In [ ]:
epochs = 50
epoch_lr = 0.1
counter = 0
test_counter = 0

print("starting cell")
sess = tf.Session()
sess.run(tf.global_variables_initializer())

checkpoint_dir = "./checkpoints"
writer = tf.summary.FileWriter("./logs", sess.graph)
saver = tf.train.Saver()

test_feed_dict = {test_input: aug_test_images, test_label: test_labels}

print("Starting Train!!!")
start_time = time.time()
for epoch in range(epochs):
    if epoch == int(epochs * 0.5) or epoch == int(epochs * 0.75):
        epoch_lr = epoch_lr * 0.1
    minibatches, n_batches = get_mini_batches(train_images, train_labels, batch_size = batch)
    i = 0
    for minibatch in minibatches:
        i += 1
        X, y = minibatch
        
        train_feed_dict = {train_input:X, train_label: y, lr: epoch_lr}
        
        acc, _, l, summary1 = sess.run([train_acc, optimizer, train_loss, train_summary], feed_dict=train_feed_dict)
        writer.add_summary(summary1, counter)
        counter += 1
        
        print("Epoch: [%2d] [%5d/%5d], train_accuracy: %.2f, learning_rate : %.4f, train_loss: %.2f" \
                      % (epoch, i, n_batches, acc, epoch_lr, l))
    if(epoch%5 == 0):
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver.save(sess, os.path.join(checkpoint_dir, 'ResNet18.model'), global_step=counter)
        
    t_acc, summary2, t_loss = sess.run([test_acc, test_summary, test_loss], feed_dict=test_feed_dict)
    writer.add_summary(summary2, test_counter)
    print("Epoch: [%2d], test_accuracy: %.2f, test_loss: %.2f"%(epoch, t_acc, t_loss))
    test_counter += 1
    
sess.close()

starting cell
Starting Train!!!
Epoch: [ 0] [    1/   59], train_accuracy: 0.16, learning_rate : 0.1000, train_loss: 4.23
Epoch: [ 0] [    2/   59], train_accuracy: 0.16, learning_rate : 0.1000, train_loss: 39.27


In [9]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

# out = sess.run(train_output, feed_dict={train_input: data_augmentation(test_images[0:15], 224)})

# print(out)
# sess.close()

In [ ]:
# mini, n = get_mini_batches(train_images[0:15], train_labels[0:15])
# for i in mini:
#     x, y = i
#     break
    
# x = np.array(x)
# y = np.array(y)

# idx = 2
# import matplotlib.pyplot as plt
# plt.imshow(x[idx])
# y[idx]